# Modélisation de l’incertitude — Master 2 MIAGE IA2
## Travaux dirigés N◦ 3 : Réseaux bayésiens


On va construire et utiliser des résaux bayésiens en se basant sur une bibliothèque Python, **Pomegranate en l’occurrence.**

### 1. Introduction
* Le but de ce TP va être de vous familiariser avec les réseaux bayésiens. Pour ce faire, nous nous baserons sur l’utilisation de Pomegranate (https://pomegranate.readthedocs.io/en/docs/index.html), un module Python qui réalise une grande variété de modèles probabilistes, parmi lesquelles les réseaux bayésiens. 

### 2. Prise en main
* 1. Construction d’un réseau : 
    * Pour prendre en main Pomegranate, nous allons étudier la situation suivante : Un arbre peut perdre ses feuilles pour deux raisons : 
        * soit il est malade, 
        * soit le sol est trop sec. 
    * Il y a a priori **10%** de chances qu’il soit malade ; de même, la probabilité que le sol soit sec est de **10%**
    * Soit P(malade) = 0.1
    * Et P(sec) = 0.1

* Les probabilités conditionnelles de perte
de feuilles en fonction de la sécheresse et de la maladie sont les suivantes :

|                  |   sec  |   sec   |  ¬sec  |   ¬sec  |
| :--------------: | :----: | :-----: | :----: | :-----: |
|                  | malade | ¬malade | malade | ¬malade |
| perd_feuilles    |  0.95  |   0.85  |  0.90  |   0.02  |
| ¬perd_feuilles   |  0.05  |   0.15  |  0.10  |   0.98  |




Créons maintenant les variables aléatoires de notre réseau 

In [1]:
from pomegranate import *
G = BayesianNetwork("BotanicalProblem")

malade = DiscreteDistribution({'Oui': 0.1, 'Non': 0.9})
sec = DiscreteDistribution({'Oui': 0.1, 'Non': 0.9})
perd_feuilles = ConditionalProbabilityTable(
    [
        ['Non', 'Non', 'Non', .98],
        ['Non', 'Non', 'Oui', .02],
        ['Non', 'Oui', 'Non', .15],
        ['Non', 'Oui', 'Oui', .85],
        ['Oui', 'Non', 'Non', .1],
        ['Oui', 'Non', 'Oui', .9],
        ['Oui', 'Oui', 'Non', .05],
        ['Oui', 'Oui', 'Oui', .95]
    ], [malade, sec])


Ensuite, il faut créer la structure du réseau, comme suit : 

In [2]:
s = State(malade, name = 'malade')
d = State(sec, name = 'sec')
l = State(perd_feuilles, name = 'perd_feuilles')
G.add_states(s, d, l)
G.add_edge(s, l)
G.add_edge(d, l)
G.bake()

In [3]:
print(G)

BotanicalProblem:{
    "class" : "State",
    "distribution" : {
        "class" : "Distribution",
        "dtype" : "str",
        "name" : "DiscreteDistribution",
        "parameters" : [
            {
                "Oui" : 0.1,
                "Non" : 0.9
            }
        ],
        "frozen" : false
    },
    "name" : "malade",
    "weight" : 1.0
}{
    "class" : "State",
    "distribution" : {
        "class" : "Distribution",
        "dtype" : "str",
        "name" : "DiscreteDistribution",
        "parameters" : [
            {
                "Oui" : 0.1,
                "Non" : 0.9
            }
        ],
        "frozen" : false
    },
    "name" : "sec",
    "weight" : 1.0
}{
    "class" : "State",
    "distribution" : {
        "class" : "Distribution",
        "name" : "ConditionalProbabilityTable",
        "table" : [
            [
                "Oui",
                "Oui",
                "Non",
                "0.05000000000000001"
            ],
            

* 2. Inférence bayésienne. On peut maintenant directement calculer les probabilités des valeurs de toutes les variables, avec la méthode predict_proba, qui prend un dictionnaire d’observations (les évidences). On commencera par un réseau sans évidences 

In [4]:
beliefs = G.predict_proba({})
beliefs = map(str, beliefs)
for state, belief in zip(G.states, beliefs):
    print(state.name, belief)

malade {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "Oui" : 0.10000000000000034,
            "Non" : 0.8999999999999996
        }
    ],
    "frozen" : false
}
sec {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "Oui" : 0.10000000000000032,
            "Non" : 0.8999999999999997
        }
    ],
    "frozen" : false
}
perd_feuilles {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "Non" : 0.8167999999999995,
            "Oui" : 0.18320000000000042
        }
    ],
    "frozen" : false
}


Vous devriez voir qu’en l’absence d’évidence, nos arbres ont
presque 82% de chances de garder leur feuilles. Maintenant, en regardant dans le jardin, vous constatez que les arbres perdent quand même leurs feuilles

In [5]:
beliefs = G.predict_proba({'perd_feuilles': 'Oui'})
beliefs = map(str, beliefs)
for state, belief in zip(G.states, beliefs):
    print(state.name, belief)

malade {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "Oui" : 0.4939956331877731,
            "Non" : 0.5060043668122269
        }
    ],
    "frozen" : false
}
sec {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "Oui" : 0.4694323144104805,
            "Non" : 0.5305676855895196
        }
    ],
    "frozen" : false
}
perd_feuilles Oui


Il y a donc 47% de chances que votre sol soit sec et 49% pour que vos arbres soient malades. Vous arrosez régulièrement vos plantation et pourtant elles perdent toujours leurs feuilles ?

In [6]:
beliefs = G.predict_proba({'perd_feuilles': 'Oui', 'sec': 'Non'})
beliefs = map(str, beliefs)
for state, belief in zip(G.states, beliefs):
    print(state.name, belief)

malade {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "Oui" : 0.8333333333333325,
            "Non" : 0.16666666666666755
        }
    ],
    "frozen" : false
}
sec Non
perd_feuilles Oui


Vous avez donc 83% de chances que vos arbres soient malades. . .

## 3. Licencié ou augmenté ?
* Dans l’entreprise Bayes&Co, il y a un patron despotique qui décide périodiquement de licencier des employés et d’augmenter le salaire d’autres employés. 
* Sa politique n’est pas très populaire car il y a **autant de licenciements que d’augmentations.**
* Chaque fois que le patron prend une de ces décisions, elle doit être traitée par le département des ressources humaines et par la comptabilité, qui envoient ensemble une notification à l’employé (mal)heureux.
* Le département des ressources humaines compte trois employés : Janet, Jim et Julia. Lors d’un **licenciement, Janet s’occupe du dossier dans 30% des cas, Jim dans 35% et Julia 35%.** 
* Si par contre il s’agit **d’une augmentation de salaire, les proportions passent à 20%, 20% et 60% respectivement.**
* À la comptabilité travaillent James et Jules. **Lors d’un licenciement, James s’occupe de 37% des cas (et Jules de 63%)**, 
* alors que lors d’une augmentation la proportion passe à **42% (et donc 58% pour Jules)**. 
* Les employés concernés peuvent être avertis au choix par mail ou par courrier.
Il suffit que les deux responsables du dossier se mettent d’accord. On constate donc que **Janet et James écrivent une lettre dans 40% des cas (et donc un mail dans 60% des cas)** ; 
* mais **lorsque Janet collabore avec Jules, ces proportions sont de 70% et 30%** ; 
* **pour Jim et James 30% et 70%** ; 
* **pour Jim et Jules 65% et 35%** ; 
* **pour Julia et James 25% et 75%** ; 
* **enfin pour Julia et Jules, 55% et 45%**


#### 2. En utilisant Pomegranate, écrivez un programme implémentant ce réseau.

* **autant de licenciements que d’augmentations.**
    * P(licenciement) = P(augmentation)
    * P(licenciement) = 0.5 (licenciement => ¬augmentation )
    * P(augmentation) = 0.5 (augmentation => ¬licenciement)


* Ressources Humaines

|       **X**      |   lencenciement |   augmentation   |             P(X)              |
| :--------------: | :-------------: | :--------------: | :---------------------------: |
|      Janet       |       0.3       |        0.2       | 0.5* 0.3+0.5* 0.2 = **0.25**  |
|      Jim         |       0.35      |        0.2       |           **0.275**           |
|      Julia       |       0.35      |        0.6       |           **0.475**           |
|      Total       |       1         |        1         |             **1**             |


* Compta

|       **X**      |   lencenciement |   augmentation   |             P(X)              |
| :--------------: | :-------------: | :--------------: | :---------------------------: |
|      James       |       0.37      |        0.42      | 0.37* 0.5+0.42* 0.5 =**0.395**|
|      Jules       |       0.63      |        0.58      |           **0.605**           |
|      Total       |       1         |        1         |             **1**             |


* Courrier

|                   |  Janet |  Janet |  Jim   |  Jim   |  Julia |  Julia |
| :---------------: | :----: | :----: | :----: | :----: | :----: | :----: |
|                   |  James |  Jules |  James |  Jules |  James |  Jules |
|      Lettre       |  0.4   |  0.7   |  0.3   |  0.35  |  0.25  |  0.55  |
|      Mail         |  0.6   |  0.3   |  0.7   |  0.65  |  0.75  |  0.45  |
|      Total        |   1    |   1    |   1    |   1    |   1    |   1    |





In [7]:
from pomegranate import *
G = BayesianNetwork("Licencié ou Augmenté")

decision = DiscreteDistribution({'aug': 0.5, 'lic': 0.5})

RH = ConditionalProbabilityTable(
    [
        ['aug', 'Janet', .2],
        ['lic', 'Janet', .3],
        ['aug', 'Jim', .2],
        ['lic', 'Jim', .35],
        ['aug', 'Julia', .6],
        ['lic', 'Julia', .35]
    ], [decision])

COMPTA = ConditionalProbabilityTable(
    [
        ['aug', 'James', .42],
        ['lic', 'James', .37],
        ['aug', 'Jules', .58],
        ['lic', 'Jules', .63],
    ], [decision])

COURRIER = ConditionalProbabilityTable(
    [
        ['Janet', 'James','Lettre', .4],
        ['Janet', 'James','Mail', .6],
        ['Janet', 'Jules','Lettre', .7],
        ['Janet', 'Jules','Mail', .3],

        ['Jim', 'James','Lettre', .3],
        ['Jim', 'James','Mail', .7],
        ['Jim', 'Jules','Lettre', .35],
        ['Jim', 'Jules','Mail', .65],
        
        ['Julia', 'James','Lettre', .25],
        ['Julia', 'James','Mail', .75],
        ['Julia', 'Jules','Lettre', .55],
        ['Julia', 'Jules','Mail', .45],

    ], [RH, COMPTA])


In [9]:
decision_state = State(decision, name = 'decision')
rh_state = State(RH, name = 'RH')
compta_state = State(COMPTA, name = 'COMPTA')
courrier_state = State(COURRIER, name = 'COURRIER')

G.add_states(decision_state, rh_state, compta_state, courrier_state)
G.add_edge(decision_state, rh_state)
G.add_edge(decision_state, compta_state)
G.add_edge(rh_state, courrier_state)
G.add_edge(compta_state, courrier_state)

G.bake()

In [10]:
print(G)

Licencié ou Augmenté:{
    "class" : "State",
    "distribution" : {
        "class" : "Distribution",
        "dtype" : "str",
        "name" : "DiscreteDistribution",
        "parameters" : [
            {
                "aug" : 0.5,
                "lic" : 0.5
            }
        ],
        "frozen" : false
    },
    "name" : "decision",
    "weight" : 1.0
}{
    "class" : "State",
    "distribution" : {
        "class" : "Distribution",
        "name" : "ConditionalProbabilityTable",
        "table" : [
            [
                "aug",
                "Jim",
                "0.2"
            ],
            [
                "aug",
                "Julia",
                "0.6"
            ],
            [
                "aug",
                "Janet",
                "0.2"
            ],
            [
                "lic",
                "Jim",
                "0.3499999999999999"
            ],
            [
                "lic",
                "Julia",
              

In [14]:
# a)  Jeremy a reçu ce matin une lettre de Janet et Jules ; 
# quelle est la probabilité qu’il s’agisse d’une lettre de licenciement ?
Res = G.predict_proba({'RH': 'Janet', 'COMPTA': 'Jules','COURRIER':'Lettre'})
Res = map(str, Res)
for state, belief in zip(G.states, Res):
    print(state.name, belief)

decision {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "aug" : 0.38032786885245917,
            "lic" : 0.6196721311475409
        }
    ],
    "frozen" : false
}
RH Janet
COMPTA Jules
COURRIER Lettre


* Jeremy a 61% de chance que ce soit une lettre de licenciement 

In [15]:
# (b) Et s’il a reçu une lettre de Jim et James ? 
Res = G.predict_proba({'RH': 'Jim', 'COMPTA': 'James','COURRIER':'Lettre'})
Res = map(str, Res)
for state, belief in zip(G.states, Res):
    print(state.name, belief)

decision {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "aug" : 0.39344262295081983,
            "lic" : 0.6065573770491802
        }
    ],
    "frozen" : false
}
RH Jim
COMPTA James
COURRIER Lettre


* Jeremy a alors 60% de chance que ce soit une lettre de licenciement

In [16]:
# (c) Si tout ce que Jeremy sait, c’est que Julia est en charge d’un dossier le concernant, 
# quels sont ses risques d’être licencié ?

Res = G.predict_proba({'RH': 'Julia'})
Res = map(str, Res)
for state, belief in zip(G.states, Res):
    print(state.name, belief)

decision {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "aug" : 0.6315789473684211,
            "lic" : 0.368421052631579
        }
    ],
    "frozen" : false
}
RH Julia
COMPTA {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "Jules" : 0.5984210526315789,
            "James" : 0.4015789473684212
        }
    ],
    "frozen" : false
}
COURRIER {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "Mail" : 0.5704736842105264,
            "Lettre" : 0.42952631578947353
        }
    ],
    "frozen" : false
}


* Dans ce cas-ci, il a 36,8% de chance d'être licencié

In [17]:
# (d) Jeremy a reçu un mail ce matin, mais il ne sait pas qui a traité son dossier. 
# Quelle probabilité a-t-il d’y lire une annonce d’augmentation de salaire ? 
Res = G.predict_proba({'Courrier': 'Mail'})
Res = map(str, Res)
for state, belief in zip(G.states, Res):
    print(state.name, belief)
    

decision {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "aug" : 0.5000000000000001,
            "lic" : 0.4999999999999999
        }
    ],
    "frozen" : false
}
RH {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "Jim" : 0.275,
            "Julia" : 0.47499999999999987,
            "Janet" : 0.25000000000000017
        }
    ],
    "frozen" : false
}
COMPTA {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "Jules" : 0.6049999999999998,
            "James" : 0.39500000000000013
        }
    ],
    "frozen" : false
}
COURRIER {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "Mail" : 0.5588437500000001,
            "Lettre" : 0.44115624999999997
        }

* Sa probabilité d'être augmenté est de 50%.

## 4. Enquête
Un cambriolage a été constaté au siège de l’entreprise BN Tech. Une enquête préliminaire a permis de montrer que le coupable a opéré seul et qu’il s’agit forcément de l’un des 100 employés que compte l’entreprise.
Quelques jours plus tard, la police a reçu une lettre anonyme accusant Pierre Kiroul, un nouvel employé de BN Tech. La police a immédiatement relevé ses empreintes digitales et les a comparé à celles retrouvées sur le lieu du forfait. Aucune trace correspondant à M. Kiroul n’a pu être mise en évidence. Mais selon l’inspecteur de la police scientifique chargé de l’affaire, il ne faudrait pas en tirer des conclusions hâtives. En effet, le vol a pu être effectué avec des gants :

* Environ 45% des cambrioleurs recourent à cette technique (alors que la probabilité de mettre
des gants sans faire de cambriolage peut être considérée comme nulle, dans ce contexte). 
* Si le voleur n’a pas mis des gants, il y a 99% de chances que nous détections ses empreintes ; 
* mais avec des gants, cette chance tombe à 2%. 
* La probabilité d’observer les empreintes de quelqu’un de non-coupable sur les lieux du crime est de 5%, estime l’expert de la police.
* À ce point-là de l’histoire (1), Pierre semblait presque hors de cause. Mais la police avait également trouvé sur le lieux du forfait quelques cheveux du coupable, ce qui lui a permis de faire une analyse ADN. Cette analyse a provoqué un retournement de situation : l’ADN du coupable correspondait à celui de Pierre ! 
* Ce test est d’une grande fiabilité ; les taux de faux positifs et de faux négatifs sont tous deux à 0.1%. Autant dire que nous avons maintenant une quasi-certitude que M. Kiroul est le coupable, triomphe l’inspecteur. (2)
* Pierre Kiroul a donc été accusé ; à son procès, son avocat a brillamment montré que Pierre était dans l’impossibilité de se fournir le type de gants nécessaire à la réalisation propre du crime dont on l’accusait. L’hypothèse que Pierre portait des gants doit donc être écartée. (3)
* M. Kiroul va-t-il être condamné ou acquitté ?


1. Représenter la situation sous forme de réseau bayésien (sur papier, avec les tables de probabilité).

* Gants et empreintes

|                  |      Gant       |       ¬Gant      |
| :--------------: | :-------------: | :--------------: |
|     Empreinte    |       0.02      |        0.99      |
|    ¬Empreinte    |       0.98      |        0.01      |
|       Total      |         1       |          1       |

* Coupable et empreintes

|                  |    Empreinte    |    ¬Empreinte    |
| :--------------: | :-------------: | :--------------: |
|     Coupable     |       0.95      |        0.0       |
|    ¬Coupable     |       0.05      |        1.0       |
|       Total      |         1       |          1       |


* Test ADN
* Faux positif = 0.01, Faux négatif = 0.01

|                  |    prédit_Coupable     |    ¬prédit_Coupable     |
| :--------------: | :--------------------: | :---------------------: |
|     Coupable     |           0.99         |          0.01           |
|    ¬Coupable     |           0.01         |          0.99           |
|       Total      |            1           |            1            |




2. En utilisant Pomegranate, écrivez un programme implémentant ce réseau.

In [3]:
from pomegranate import *

G = BayesianNetwork("CRIME")
Gants = DiscreteDistribution({"Gants": 0.45, "Sans Gants": 0.55})


Empreinte = ConditionalProbabilityTable(
    [
        ["Gants", "Empreinte", 0.02],
        ["Gants", "Sans Empreinte", 0.98],
        ["Sans Gants", "Empreinte", 0.99],
        ["Sans Gants", "Sans Empreinte", 0.01]
    ],
    [Gants]
)


COUPABLE = ConditionalProbabilityTable(
    [
        ["Empreinte", "Coupable", 0.95],
        ["Empreinte", "Non Coupable", 0.05],
        ["Sans Empreinte", "Coupable", 0.0],
        ["Sans Empreinte", "Non Coupable", 1.0]

    ],
    [Empreinte]
)


ADN = ConditionalProbabilityTable(
    [
        ["Coupable","predit Coupable", 0.99],
        ["Coupable","predit Non Coupable", 0.01],
        ["Non Coupable","predit Coupable", 0.01],
        ["Non Coupable","predit Non Coupable", 0.99]
    ],
    [COUPABLE]
)

gant_state = State(Gants, name="Gants")
empreinte_state = State(Empreinte, name="Empreinte")
coupable_state = State(COUPABLE, name="Coupable")
adn_test = State(ADN, name="Test ADN")

G.add_states(gant_state,empreinte_state,coupable_state,adn_test)
G.add_edge(gant_state, empreinte_state)
G.add_edge(empreinte_state, coupable_state)
G.add_edge(coupable_state, adn_test)

G.bake()


3. Utiliser cette implémentation pour calculer la probabilité de culpabilité de Pierre Kiroul
aux points (1), (2) et (3) de l’histoire ci-dessus.
Rendez votre code, vos réponses aux questions et vos observations dans un archive zippé par
courriel.

(1) Probabilité d'être coupable : 

In [4]:
Res = G.predict_proba({'COUPABLE': 'Coupable'})
Res = map(str, Res)
for state, belief in zip(G.states, Res):
    if state.name == "Coupable":
        print(state.name, belief)

Coupable {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "Non Coupable" : 0.47417500000000007,
            "Coupable" : 0.525825
        }
    ],
    "frozen" : false
}


La probabilité qu'il soit coupable à l'étape 1 est de 52,6 %

(2) Probabilité du test adn

In [8]:
Res = G.predict_proba({'ADN': 'predit Coupable', "COUPABLE": "Coupable"})
Res = map(str, Res)
for state, belief in zip(G.states, Res):
    print(state.name, belief)

Gants {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "Gants" : 0.4500000000000001,
            "Sans Gants" : 0.55
        }
    ],
    "frozen" : false
}
Empreinte {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "Empreinte" : 0.5535,
            "Sans Empreinte" : 0.4465000000000002
        }
    ],
    "frozen" : false
}
Coupable {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "Non Coupable" : 0.47417500000000007,
            "Coupable" : 0.525825
        }
    ],
    "frozen" : false
}
Test ADN {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "predit Non Coupable" : 0.47469150000000004,
            "predit Coupable" : 0.5253085000000001
        }
  

Le test ADN a changé de valeur pour la prédiction 

(3) Probabilité qu'il portait des gants :


In [9]:
Res = G.predict_proba({'ADN': 'predit Coupable', "COUPABLE": "Coupable", "Gants" : "Sans Gants"})
Res = map(str, Res)
for state, belief in zip(G.states, Res):
    print(state.name, belief)

Gants Sans Gants
Empreinte {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "Empreinte" : 0.9899999999999995,
            "Sans Empreinte" : 0.01000000000000044
        }
    ],
    "frozen" : false
}
Coupable {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "Non Coupable" : 0.05950000000000061,
            "Coupable" : 0.9404999999999993
        }
    ],
    "frozen" : false
}
Test ADN {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "predit Non Coupable" : 0.06831000000000059,
            "predit Coupable" : 0.9316899999999995
        }
    ],
    "frozen" : false
}


Sans gant, il est coupable à 94 % 

In [10]:
Res = G.predict_proba({'ADN': 'predit Coupable', "COUPABLE": "Coupable", "Gants" : "Gants"})
Res = map(str, Res)
for state, belief in zip(G.states, Res):
    print(state.name, belief)

Gants Gants
Empreinte {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "Empreinte" : 0.020000000000000438,
            "Sans Empreinte" : 0.9799999999999995
        }
    ],
    "frozen" : false
}
Coupable {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "Non Coupable" : 0.9809999999999993,
            "Coupable" : 0.01900000000000064
        }
    ],
    "frozen" : false
}
Test ADN {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "predit Non Coupable" : 0.9713799999999994,
            "predit Coupable" : 0.028620000000000635
        }
    ],
    "frozen" : false
}


Avec gants, il est coupable à 1%